In [72]:
import requests
import json
from datetime import datetime
from md2notion_blocks import markdown_to_notion_blocks
import requests
import json
import pandas as pd

In [73]:
def sheet_payload(df, raw_cols, n=30, date_col="date"):
    df = df.sort_values(date_col)
    df[date_col] = pd.to_datetime(df[date_col])

    last = df.iloc[-1]
    as_of = last[date_col].strftime("%Y-%m-%d")

    latest = {
        c: float(last[c])
        for c in df.columns if c != date_col and pd.notna(last[c])
    }

    raw = {
        c: [
            [d.strftime("%Y-%m-%d"), float(v)]
            for d, v in zip(df[date_col].tail(n), df[c].tail(n))
            if pd.notna(v)
        ]
        for c in raw_cols if c in df.columns
    }

    return {"as_of": as_of, "latest_metrics": latest, "raw_series": raw}


def xlsx_to_payload(path, raw_map, n=30):
    xls = pd.ExcelFile(path)
    return {
        "meta": {"as_of": datetime.today().strftime("%Y-%m-%d")},
        "indicators": {
            s: sheet_payload(
                pd.read_excel(xls, s),
                raw_map.get(s, []),
                n
            )
            for s in xls.sheet_names
        }
    }


In [74]:
path = r"C:\Users\angel.fan\Downloads\reports_output (2) (1).xlsx"

raw_map = {"rrp": ["RRP_balance"],
           "srf": ["SRF_balance"],
           "walcl": ["walcl"],
           "tga": ["tga"],
           "ig":["IG_OAS"],
           "hy":["HY_OAS"],
           "se":["SOFR-EFFR"],
           "stress":["stress_index"]}

out = xlsx_to_payload(path, raw_map, n=30)
with open("llm_payload.json", "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)
json_text = json.dumps(out, ensure_ascii=False, indent=2)


In [75]:
API_KEY = "sk-5d151ba715db4c9b85ba872433bcea3a"
API_URL = "https://api.deepseek.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "deepseek-chat",
    "messages": [
        {"role": "system", "content": """使用用户上传的JSON数据{json_text}，生成《每日流动性与市场压力日报》。请基于数据完成任务，输出日报的格式为markdown。日报必须严格按照固定结构输出，分为两个主题、共七个自然段，每段150–200字，逻辑连贯、分析精准，不罗列数据、不定义指标。

结构如下：

一、资金供应端（共三段）
1. RRP（Reverse Repo Balance），单位为“亿美元”，分析余额变化及其对流动性松紧的信号，注明日期。注意：RRP 的原始数据单位为“十亿美元”，若数值为 6，应换算为“60 亿美元”。
2. SRF（Standing Repo Facility），单位为“亿美元”，分析银行体系短期资金压力与流动性需求，注明日期。注意：SRF 的原始数据单位为“十亿美元”，若数值为 6，应换算为“60 亿美元”。
3. WALCL（美联储资产负债表总资产）与 TGA（财政部一般账户）合并分析，单位为“百万美元”，须在文中明确说明其为周度指标，结合资产负债表扩缩与财政收支对流动性投放或回笼的综合影响，注明日期。

二、市场压力端（共四段）
1. HY-Spread（高收益债利差），单位%，分析市场风险偏好变化。
2. IG-Spread（投资级债利差），单位%，评估机构风险定价与市场稳定性。
3. SOFR-Spread（SOFR–EFFR 利差），单位%，观察短端资金供需变化与钱荒风险。
4. Stress Index（金融市场压力指数），判断系统性金融压力。

写作要求：
- 每段150–200字，语气稳健、专业；
- 两个主题之间以主题名称作分隔行（如“【资金供应端】”“【市场压力端】”），其余部分不出现小标题；
- WALCL与TGA为周度指标，须在对应段落中明确说明；
- 所有涉及日期的分析必须明确说明时间区间，例如“2025年11月27日至11月28日”；
- 若仅提供单日数据，系统自动与前一日或上一期比较并注明时间；
- 分析必须结合历史时间序列数据的走势，重点参考过去一周至一月的连续数据点，识别趋势变化与波动模式，不能仅依赖均值、分位、波动率、z-score等静态指标；
- 长期趋势判断可结合年内时间序列演变轨迹，明确指出当前数值处于全年分布的相对位置；
- 不得机械罗列或解释技术指标定义，而应将其用于支撑逻辑判断；
- 避免使用模糊时间表述，如“近日”“近期”“近几天”；
- 不得使用模糊结论或“建议式”表达，如“仍需观察”“反映出……之间的拉锯”“建议继续观测”等；
- 分析必须以数据变化为逻辑主线，结论落点明确，例如“显示流动性边际收紧”或“反映资金面趋稳”；
- 语气冷静、客观、直接，不使用不确定或空洞性语句；
- 引用当日数据并注明日期与单位；
- 根据变动幅度判断重点；
- 若数据缺失，以谨慎语气逻辑补充；
- 输出仅包含上述两部分共七段正文，格式整洁规范，可直接用于日报发布；
- 所有金额单位应使用中文表达：“亿美元”代表十亿级，“百万美元”代表百万级，不使用Bn或Mln英文缩写；系统应根据段落设定单位进行统一换算和表述，不得擅自扩大或缩小数量级；
- 严格将 billion 解析为 10 亿（即 1B = 10^9），不得错误理解为 hundred billion 或乘以100，不得将如“7.561B”错误换算为“7561 亿美元”，应为“75.61 亿美元”；
- RRP 和 SRF 数据单位原始为“十亿美元”，所有文本输出必须乘以10后以“亿美元”为单位表达，并精确至0.01亿；
- 模型必须明确知悉：输入数据中未直接标注“B”或“M”等单位，单位解释完全依据段落标准进行：1）RRP（Reverse Repo Balance）为“十亿美元”，2）SRF（Standing Repo Facility）为“十亿美元”，3）WALCL（美联储资产负债表总资产）与TGA（财政部一般账户）为“百万美元”。所有换算必须依据该规则，不得根据数字本身作出单位推测；
- 模型必须严格引用表格中明确列示的数据日期（data date），不得以系统日期代替；当表格日期与自然日不一致时，以表格中所示数据日为主，分析中须明确标注对比的两个具体日期；
- 严禁将周末或节假日误判为数据日期，若表格所示为周末，则应明确识别其实际所代表的前一个工作日数据，所有日期引用须精准对应真实交易或发布日；
- 若表中日期为周末或节假日，则仅用于展示目的，不得视作有效数据点，必须回溯至上一个真实交易日数据并据此撰写分析，所有引用与对比日期均须为有效数据时间点，不能伪造非交易日数据。"""}
    ],
    "temperature": 0.3
}

response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
result = response.json()

print(result["choices"][0]["message"]["content"])

【资金供应端】

隔夜逆回购（RRP）余额在2025年11月27日为75.61亿美元，较前一日的75.61亿美元持平。这一水平在过去一周内保持稳定，与11月中旬以来持续低于百亿美元的状况一致。回顾近一个月的时间序列，RRP余额自10月末从约200亿美元水平持续回落，目前已稳定在极低区间。该余额的持续低位运行，清晰地表明金融体系内过剩流动性已基本被吸收，银行间市场对美联储隔夜逆回购工具这一“流动性蓄水池”的需求已降至冰点。当前余额处于全年绝对低位，远低于年初动辄数千亿美元的规模，这标志着由量化紧缩（QT）和财政部发债共同驱动的流动性收缩进程已进入新阶段，市场整体流动性环境已从充裕转向紧平衡。

常备回购便利（SRF）的使用额在2025年11月27日为0.00亿美元，与前一交易日相同，且已连续多日维持零使用。从过去一个月的数据轨迹观察，SRF仅在11月初个别交易日有微量使用，其余时间均处于闲置状态。这一现象直接反映出，在当前的货币政策与市场环境下，符合资格的存款机构能够通过常规的联邦基金市场等渠道满足其短期资金需求，并未面临迫切的、需要通过SRF这一最后贷款人工具来获取流动性的压力。SRF的持续零使用，是银行体系短期资金压力可控、市场内部融资渠道畅通的明确信号，也侧面印证了尽管RRP余额极低，但尚未引发广泛的、结构性的流动性短缺。

截至2025年11月26日当周（此为周度指标），美联储资产负债表总资产（WALCL）为7,561,000百万美元，较前一周的7,561,000百万美元无变化；同期，财政部一般账户（TGA）余额为550,000百万美元，亦与前一周持平。将两者合并审视，资产负债表规模维持稳定，意味着美联储通过公开市场操作进行的流动性投放或回笼处于中性状态。而TGA余额在550,000百万美元的高位企稳，表明财政部仍在维持较大的现金缓冲。这一组合对市场流动性的综合影响趋于中性偏紧：WALCL的稳定意味着美联储未主动扩表提供增量流动性，而高企的TGA余额则持续从银行体系抽离资金并存放在央行，构成了事实上的流动性回笼。这一格局若持续，将继续对市场整体流动性构成温和的收缩压力。

【市场压力端】

高收益债利差（HY-Spread）在2025年11月27日为5.50%，较前一日的5.50%保持稳定。纵观过去一周，该利差在5.45%至5.55%的狭窄区间内波动，并未出现

In [80]:
markdown_text = result["choices"][0]["message"]["content"]
blocks = markdown_to_notion_blocks(markdown_text)

In [ ]:
NOTION_TOKEN = "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"
DATABASE_ID = "2ca74daad637803fa61cf006e907b5eb"

url = "https://api.notion.com/v1/pages"

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

payload = {
    "parent": {
        "database_id": DATABASE_ID
    },
    "properties": {
        # Title 属性（必须）
        "Name": {
            "title": [
                {
                    "text": {
                        "content": "Liquidity (Day) - {today}"
                    }
                }
            ]
        },

        # Multi-select / Tag
        "Tags": {
            "multi_select": [
                {"name": "AI报告"},
                {"name": "流动性报告"}
            ]
        },

        # Date
        "Report Date": {
            "date": {
                "start": datetime.now().strftime("%Y-%m-%d")
            }
        }
    },
    "children": blocks
            
}

response = requests.post(url, headers=headers, data=json.dumps(payload))

print(response.status_code)
print(response.json())

200
{'object': 'page', 'id': '2cb74daa-d637-81b5-bb35-e5b821239412', 'created_time': '2025-12-16T08:38:00.000Z', 'last_edited_time': '2025-12-16T08:38:00.000Z', 'created_by': {'object': 'user', 'id': 'a0af6375-070e-49b3-af04-fea91bfe747f'}, 'last_edited_by': {'object': 'user', 'id': 'a0af6375-070e-49b3-af04-fea91bfe747f'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '2ca74daa-d637-803f-a61c-f006e907b5eb'}, 'archived': False, 'in_trash': False, 'is_locked': False, 'properties': {'Tags': {'id': '%5DrKg', 'type': 'multi_select', 'multi_select': [{'id': '09e74f0a-77e2-4bf3-b437-3bb387b5d198', 'name': 'AI报告', 'color': 'yellow'}, {'id': '9157880b-3893-41a1-8072-7afe94b8c25d', 'name': '流动性报告', 'color': 'red'}]}, 'Report Date': {'id': '%5ELQW', 'type': 'date', 'date': {'start': '2025-12-16', 'end': None, 'time_zone': None}}, 'Name': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Liquidity (Day) - {today}', 'link': None}, 'anno

In [78]:
url = f"https://api.notion.com/v1/databases/2ca74daad637803fa61cf006e907b5eb"
response = requests.get(url, headers=headers)

print(response.status_code)
print(response.json())

200
{'object': 'database', 'id': '2ca74daa-d637-803f-a61c-f006e907b5eb', 'cover': None, 'icon': None, 'created_time': '2025-12-15T02:36:00.000Z', 'created_by': {'object': 'user', 'id': '2b9d872b-594c-813e-9f81-0002c8616fdf'}, 'last_edited_by': {'object': 'user', 'id': '2b9d872b-594c-813e-9f81-0002c8616fdf'}, 'last_edited_time': '2025-12-16T04:04:00.000Z', 'title': [{'type': 'text', 'text': {'content': 'Liquidity Report', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Liquidity Report', 'href': None}], 'description': [], 'is_inline': False, 'properties': {'Tags': {'id': '%5DrKg', 'name': 'Tags', 'description': None, 'type': 'multi_select', 'multi_select': {'options': [{'id': '09e74f0a-77e2-4bf3-b437-3bb387b5d198', 'name': 'AI报告', 'color': 'yellow', 'description': None}, {'id': '9157880b-3893-41a1-8072-7afe94b8c25d', 'name': '流动性报告', 'color': 'red', 'description': None}]}}, 'Rep

In [79]:
json_text

'{\n  "meta": {\n    "as_of": "2025-12-16"\n  },\n  "indicators": {\n    "rrp": {\n      "as_of": "2025-12-12",\n      "latest_metrics": {\n        "RRP_balance": 0.838,\n        "RRP_change_prev_point": -2.036,\n        "RRP_pectage_change_prev_point": -0.708420320111343,\n        "RRP_ma_7d": 2.7342,\n        "RRP_ma_14d": 2.8763,\n        "RRP_ma_30d": 2.927666666666667,\n        "RRP_percentile_1y": 0.004\n      },\n      "raw_series": {\n        "RRP_balance": [\n          [\n            "2025-10-30",\n            19.166\n          ],\n          [\n            "2025-10-31",\n            51.802\n          ],\n          [\n            "2025-11-03",\n            23.792\n          ],\n          [\n            "2025-11-04",\n            16.983\n          ],\n          [\n            "2025-11-05",\n            12.814\n          ],\n          [\n            "2025-11-06",\n            10.754\n          ],\n          [\n            "2025-11-07",\n            4.903\n          ],\n          